<a href="https://colab.research.google.com/github/sravanisasu/Aganitha/blob/master/Binary_Classification_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Morlet Function**

In [3]:
# Morlet Function
def w_func(t):
  return torch.cos(1.75*t)* torch.exp(-(t**2))

**Required imports**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import weight_norm
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import minmax_scale  
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix, classification_report

**Pre-process the data**

In [ ]:
df = pd.read_csv("/content/Dataset_spine.csv")

# applying encoding for the output 0-normal and 1-abnormal
sns.countplot(x = 'Class_att', data=df)
df['Class_att'] = df['Class_att'].astype('category')
encode_map = {
    'Abnormal': 1,
    'Normal': 0
}
df['Class_att'].replace(encode_map, inplace=True)

X = df.iloc[:, 0:-1]
y = df.iloc[:, -1]

# split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=69)

# apply minmax scalar
X_train = minmax_scale(X_train)
X_test = minmax_scale(X_test)

**Initalise Hyper parameters**

In [ ]:
nin = len(X.columns)
nhn = 10
non = 1

EPOCHS = 500
BATCH_SIZE = 64
LEARNING_RATE = 0.001

**Function for train dataset**

In [ ]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
        
    def __len__ (self):
        return len(self.X_data)

**Function for test dataset**

In [ ]:
## test data    
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)

**Load Dataset**

In [ ]:
#train
train_data = trainData(torch.FloatTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

#test
test_data = testData(torch.FloatTensor(X_test))
test_loader = DataLoader(dataset=test_data, batch_size=1)

**Define Model**

In [ ]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        self.fc1 = weight_norm(nn.Linear(nin, nhn, bias=False))
        self.fc2 = nn.Linear(nhn, non)
        self.a = nn.Parameter(torch.rand(nhn), requires_grad=True)
        self.b = nn.Parameter(torch.rand(nhn), requires_grad=True)
    def forward(self, x):
        t = (self.fc1(x)-self.b)/self.a
        vk = self.fc2(w_func(t))
        return vk

**Define accuracy function**

In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#define model
model = binaryClassification()
model.to(device)

#define loss function
criterion = nn.BCEWithLogitsLoss()

#define optimiser
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

#train model
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

# evaluate or predit the results of model
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

print(auc(y_test, y_pred_list))

Epoch 001: | Loss: 0.71980 | Acc: 38.500
Epoch 002: | Loss: 0.73254 | Acc: 29.750
Epoch 003: | Loss: 0.72016 | Acc: 34.750
Epoch 004: | Loss: 0.71899 | Acc: 40.250
Epoch 005: | Loss: 0.69995 | Acc: 48.000
Epoch 006: | Loss: 0.71009 | Acc: 41.250
Epoch 007: | Loss: 0.70070 | Acc: 48.500
Epoch 008: | Loss: 0.69631 | Acc: 54.250
Epoch 009: | Loss: 0.69033 | Acc: 55.750
Epoch 010: | Loss: 0.69230 | Acc: 52.000
Epoch 011: | Loss: 0.69054 | Acc: 58.000
Epoch 012: | Loss: 0.68921 | Acc: 57.250
Epoch 013: | Loss: 0.69047 | Acc: 57.750
Epoch 014: | Loss: 0.68590 | Acc: 60.500
Epoch 015: | Loss: 0.68567 | Acc: 60.250
Epoch 016: | Loss: 0.68230 | Acc: 61.750
Epoch 017: | Loss: 0.68399 | Acc: 62.500
Epoch 018: | Loss: 0.68321 | Acc: 60.750
Epoch 019: | Loss: 0.67711 | Acc: 64.500
Epoch 020: | Loss: 0.67540 | Acc: 67.250
Epoch 021: | Loss: 0.67613 | Acc: 66.000
Epoch 022: | Loss: 0.67013 | Acc: 70.750
Epoch 023: | Loss: 0.67640 | Acc: 65.500
Epoch 024: | Loss: 0.67250 | Acc: 66.250
Epoch 025: | Los

ValueError: ignored